In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

class_name = "bone"
dataset_dir = f"{class_name}_dataset_3"
mask_gt_dir = f"valid_mask"

model = "mask_rcnn_R_50_FPN_3x"
# model = "mask_rcnn_R_101_FPN_3x"
# model = "mask_rcnn_R_50_C4_1x"
# model = "mask_rcnn_R_50_C4_3x"
# model = "segformer"
# model = "DeepLab(MobileNetv3)"
mask_pred_dir = f"valid_results_instance({model})"

In [ ]:
print(len(os.listdir(f"./detectron2/AHE_dataset/{dataset_dir}/{mask_gt_dir}")))
print(len(os.listdir(f"./detectron2/AHE_dataset/{dataset_dir}/{mask_pred_dir}")))

## 刪除不要的 mask 類別 ，記得要改路徑跟要保留的類別

In [ ]:
mask_dir = f"./detectron2/AHE_dataset/{dataset_dir}/{mask_pred_dir}" # !!!!!!!!!!!

remove_count = 0
for name in os.listdir(mask_dir):
    mask_path = os.path.join(mask_dir, name)
    mask = cv2.imread(mask_path)
    split_name = name.split("_")

    if class_name not in split_name or mask.shape != (640, 640, 3):
        # os.remove(mask_path)
        print("remove : ", mask_path)
        remove_count += 1

print("remove count: ", remove_count)
print(len(os.listdir(mask_dir)))

## 更改檔名

In [ ]:
image_dir = f"./detectron2/AHE_dataset/{dataset_dir}/{mask_pred_dir}"

for name in os.listdir(image_dir):
    temp_list = name.split("_")
    old_path = os.path.join(image_dir, name)
    name = name.replace("_result", "")
    name = name.replace(f"_{class_name}", "")
    new_path = os.path.join(image_dir, name)
    new_path = new_path.replace(".png", ".png")

    # os.rename(old_path, new_path)
    print(old_path)
    print(new_path)

In [ ]:
annotation_mask_dir = f"./detectron2/AHE_dataset/{dataset_dir}/{mask_gt_dir}"
prediction_mask_dir = f"./detectron2/AHE_dataset/{dataset_dir}/{mask_pred_dir}"
# output_PV_mask_dir = "./detectron2/AHE_dataset/bone_dataset_2/PA_mask"
# os.makedirs(output_PV_mask_dir, exist_ok=True)

In [ ]:
print(len(os.listdir(annotation_mask_dir)))
print(len(os.listdir(prediction_mask_dir)))
# for mask_name in os.listdir(annotation_mask_dir):
#   if mask_name not in os.listdir(prediction_mask_dir):
#     print(mask_name)

## 計算每張照片的DSC、跟全部的DSC平均

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

def dice_coefficient(true_mask, pred_mask):
    intersection = np.logical_and(true_mask, pred_mask)
    return 2.0 * np.sum(intersection) / (np.sum(true_mask) + np.sum(pred_mask))

def jaccard_index(true_mask, pred_mask):
    intersection = np.logical_and(true_mask, pred_mask)
    union = np.logical_or(true_mask, pred_mask)
    return np.sum(intersection) / np.sum(union)

def mask_iou(true_mask, pred_mask):
    intersection = np.logical_and(true_mask, pred_mask).sum()
    union = np.logical_or(true_mask, pred_mask).sum()
    return 1.0 if union == 0 else intersection / union

# ---- Per-class Metrics ----
def per_class_pixel_accuracy(true_mask, pred_mask, num_classes):
    acc_list = []
    for cls in range(num_classes):
        cls_true = (true_mask == cls)
        cls_pred = (pred_mask == cls)
        correct = np.sum(cls_true & cls_pred)
        total = np.sum(cls_true)
        acc = correct / total if total > 0 else np.nan
        acc_list.append(acc)
    return acc_list

def per_class_precision(true_mask, pred_mask, num_classes):
    precision_list = []
    for cls in range(num_classes):
        cls_true = (true_mask == cls)
        cls_pred = (pred_mask == cls)
        tp = np.sum(cls_true & cls_pred)
        fp = np.sum(~cls_true & cls_pred)
        precision = tp / (tp + fp) if (tp + fp) > 0 else np.nan
        precision_list.append(precision)
    return precision_list

def per_class_recall(true_mask, pred_mask, num_classes):
    recall_list = []
    for cls in range(num_classes):
        cls_true = (true_mask == cls)
        cls_pred = (pred_mask == cls)
        tp = np.sum(cls_true & cls_pred)
        fn = np.sum(cls_true & ~cls_pred)
        recall = tp / (tp + fn) if (tp + fn) > 0 else np.nan
        recall_list.append(recall)
    return recall_list

'''----------------------------------------------------------------------------------'''
# 創建 DataFrame
results = pd.DataFrame()

for mask_name in os.listdir(annotation_mask_dir):

    if mask_name not in os.listdir(prediction_mask_dir):
        print(mask_name, " not in prediction dir")
        continue

    annotation_image = cv2.imread((annotation_mask_dir + "/" + mask_name), cv2.IMREAD_GRAYSCALE)
    prediction_image = cv2.imread((prediction_mask_dir + "/" + mask_name), cv2.IMREAD_GRAYSCALE)

    # 調整預測遮罩大小
    height, width = annotation_image.shape
    prediction_image = cv2.resize(prediction_image, (width, height), interpolation=cv2.INTER_NEAREST)

    num_classes = int(max(annotation_image.max(), prediction_image.max()) + 1)

    # DSC、Jaccard、IoU (針對整體非背景區域計算)
    dsc = round(dice_coefficient(annotation_image > 0, prediction_image > 0), 4)
    jaccard = round(jaccard_index(annotation_image > 0, prediction_image > 0), 4)
    iou = round(mask_iou(annotation_image > 0, prediction_image > 0), 4)

    # Per-class metrics
    acc_list = per_class_pixel_accuracy(annotation_image, prediction_image, num_classes)
    prec_list = per_class_precision(annotation_image, prediction_image, num_classes)
    recall_list = per_class_recall(annotation_image, prediction_image, num_classes)

    row_data = {
        'Image Name': mask_name,
        'DSC': dsc,
        'Jaccard Index': jaccard,
        'IoU': iou
    }

    # 動態加上 per-class 欄位
    for cls in range(num_classes):
        row_data[f'Acc_Class{cls}'] = round(acc_list[cls], 4) if not np.isnan(acc_list[cls]) else None
        row_data[f'Prec_Class{cls}'] = round(prec_list[cls], 4) if not np.isnan(prec_list[cls]) else None
        row_data[f'Recall_Class{cls}'] = round(recall_list[cls], 4) if not np.isnan(recall_list[cls]) else None

    results = pd.concat([results, pd.DataFrame([row_data])], ignore_index=True)

'''----------------------------------------------------------------------------------'''
# 計算平均
average_row = {'Image Name': 'Average'}
for col in results.columns:
    if col != 'Image Name':
        average_row[col] = round(results[col].mean(skipna=True), 4)

results = pd.concat([results, pd.DataFrame([average_row])], ignore_index=True)

'''----------------------------------------------------------------------------------'''
# 輸出 Excel
excel_file = f'./detectron2/AHE_dataset/{dataset_dir}/AHE_{class_name}_3_dsc({model})_perclass.xlsx'
results.to_excel(excel_file, index=False)

print("Results have been written to", excel_file)
